# Notion Database Read

## Libraries

In [1]:
# For Data Exploraty 
import pandas as pd

#For File
import json
import requests

# For Notion
from notion_client import Client

#For API Key
import os
from dotenv import load_dotenv

#API keys hidden for security !!!
load_dotenv()

# Warning Signs
import warnings
warnings.filterwarnings('ignore')

## API Configuration

In [2]:
# Token is Your API 
notion_token = os.getenv("notion_token")

# Notion URL
notion_page_id = os.getenv("notion_page_id")

#Notion Database ID
notion_database_id = os.getenv("notion_database_id")

## Notion Integration

In [3]:
headers = {
    "Authorization": "Bearer " + notion_token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

def get_pages(num_pages=None):
    """
    If num_pages is None, get all pages, otherwise just the defined number.
    """
    url = f"https://api.notion.com/v1/databases/{notion_database_id}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    payload = {"page_size": page_size}
    response = requests.post(url, json=payload, headers=headers)

    data = response.json()


    results = data["results"]
    while data["has_more"] and get_all:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{notion_database_id}/query"
        response = requests.post(url, json=payload, headers=headers)
        data = response.json()
        results.extend(data["results"])

    return results

def write_dict_to_file_as_json(content, file_name):
    content_as_json_str = json.dumps(content)

    with open(file_name, 'w') as f:
        f.write(content_as_json_str)


def safe_get(data, dot_chained_keys):

    keys = dot_chained_keys.split('.')
    for key in keys:
        try:
            if isinstance(data, list):
                data = data[int(key)]
            else:
                data = data[key]
        except (KeyError, TypeError, IndexError):
            return None
    return data


def main():
    client = Client(auth=notion_token)
    
    db_info = client.databases.retrieve(database_id=notion_database_id)

    write_dict_to_file_as_json(db_info, 'db_info.json')

    simple_rows = []

    for row in get_pages():
        words = safe_get(row, 'properties.Words.title.0.plain_text')
        description = safe_get(row, 'properties.Description.rich_text.0.text.content')

        simple_rows.append({
            'Words': words,
            'Description': description
        })

    write_dict_to_file_as_json(simple_rows, 'simple_rows.json')


if __name__ == '__main__':
    main()

# Data Explaration

In [4]:
data = pd.read_json("simple_rows.json")
data

,Words,Description
0,Certified Public Accountant (CPA),An individual who has met certain criteria and...
1,Revenue Recognition Principle,The principle that companies recognize revenue...
2,Treasurer,Employee responsible for the management of a c...
3,International Financial Reporting Standards (I...,"Accounting Standards, issued by the IASB, that..."
4,Liquidity Ratio,Measures of the short-term ability of the comp...
...,...,...
155,Weighted-average Unit Cost,Average cost that is weighted by the number of...
156,Accrued Expenses,Expenses incurred but not yet paid cash or rec...
157,Contra Revenue Account,An account that is offset against a revenue ac...
158,Statements of Stockholders Equity,A financial statement that presents the causes...
